References:
 - [1] https://en.wikipedia.org/wiki/EdDSA
 - [2] https://github.com/warner/python-pure25519

This notebook shows how SPA (Simple "Power" Analysis) attacks can work on EdDSA [1] that is implemented with weak double and add (i.e. dependent on secret bits) implementation. 
It is simulated in software by writing the execution of doubles and adds to a file.
The same could be done in a software implementation if the double and add operations can be traced in memory for example, or with side channel traces if the doubles and adds have distinct patterns.


# Attacking keypair generation
Generating the keypair is done as following:
```
seed  <= urandom(32)
a     <= sha512(seed)[:32]
A     <= Base.scalarmult(a)
pkey  <= A
```
Tracing `scalarmult` will result in recovering `a % L`, where `L` is the "modulus of the group".

This does not recover the original seed, which is needed to properly forge signatures. 
For properly forging signatures the second half of `hash(seed)` is needed (`b` below). 
However, we can still generate signatures that will pass the verification step.

# Attacking signing
A message is signed as following:
```
msg   <= <some message bytestring>
b     <= sha512(seed)[32:]
r     <= sha512(b + msg)
R     <= Base.scalarmult(r)
x     <= Hint(R + pk + msg)
S     <= (r + x * a)
sig   <= (R, S)
```
Tracing `scalarmult` will result in recovering `r % L`. With this we can recover `a % L`:
```
a     <= ((S - r) * modinv(x, L)) % L
```
As we don't have `b`, forged signatures can be detected by the fact that signing the same `msg` with the proper algorithm will result in a different signature value. 
However, the verification step will still pass.

# Example
I used [2] to understand how the steps work, and to generate the SPA trace.

In [18]:
from pure25519 import _ed25519 as raw
from pure25519.basic import (bytes_to_clamped_scalar,
                             bytes_to_scalar, scalar_to_bytes,
                             bytes_to_element, Base, L)
from pure25519.eddsa import (H, Hint)
import pure25519.config

import re, os

In [19]:
def parse_da(trace):
    trace_bin = ''.join(['1' if x == 'DA' else '0' for x in re.findall(r'(DA|D)', trace)])
    return trace_bin

def reverse_publickey(sk):
    return Base.scalarmult(sk)

def reverse_sign(S, r, pk, m):
    R = Base.scalarmult(r)
    R_bytes = R.to_bytes()
    x = Hint(R_bytes + pk + m)

    return ((S-r) * pow(x, -1, L)) % L

def forge_sign(m,a,pk):
    # We cheat by not using using sha512(skey)[32:] (the "b"), as we don't have it. This
    # means that sha512(m) is signed, instead of sha512(b + m). Signatures will not be 
    # the same for a signature made with the seed, but verification will still pass. So
    # the attack can be detected by comparing two signatures on the same message.

    # h = H(sk[:32])
    # a_bytes, inter = h[:32], h[32:]
    # a = bytes_to_clamped_scalar(a_bytes)
    r = Hint(m) #r = Hint(inter + m)
    R = Base.scalarmult(r)
    R_bytes = R.to_bytes()
    x = Hint(R_bytes + pk + m)
    S = (r + x * a)

    return R_bytes + scalar_to_bytes(S) + m

In [20]:
pure25519.config.init()
pure25519.config.spa = True
pure25519.config.spa_file = open('data/spa.txt', 'w')

# skey = bytes.fromhex('469c0b3e1abb7f8b98896876b1cdc8eb546c6d30d7c0d159e007a5466a250d55')
skey = os.urandom(32)

print('secret key:', skey.hex())
# Trace this to generate a trace for `Base.scalarmult(a)`
pkey,point = raw.publickey(skey)
print('public key:', pkey.hex())
print('')

msg = \
    b"Lorem ipsum dolor sit amet, consectetur adipiscing "\
    b"elit. Nam eget vulputate dui, eget faucibus neque. "\
    b"Mauris semper interdum nulla, quis tristique dolor "\
    b"ultricies a. Sed placerat sagittis tristique. Nam "\
    b"leo sem, pharetra non tempor sed, aliquam ac arcu. "\
    b"Curabitur consequat nisl ac urna consequat tempus. "\
    b"Ut nec diam iaculis, dignissim nunc et, dignissim "\
    b"mauris. Nunc dapibus sit amet risus id rutrum. Nullam "\
    b"ex lorem, hendrerit sed felis sed, aliquet blandit "\
    b"ligula. Curabitur scelerisque leo ut sem aliquam "\
    b"accumsan ac vitae urna."

# Trace this to generate a trace for `Base.scalarmult(r)`
tmp = raw.sign(msg, skey + pkey)
R_bytes  = tmp[:32]
S        = tmp[32:64]
msg_orig = tmp[64:]

print('R_bytes:  ', R_bytes.hex())
print('sig:      ', S.hex())
# print('msg_orig: ', msg_orig)
print('')

pure25519.config.spa = False
pure25519.config.spa_file.close()

# open() raises an exception if there's a problem, and otherwise returns the msg
msg_prime = raw.open(R_bytes + S + msg_orig, pkey)
print('msg_prime:', msg_prime)

secret key: 677e20fecbc83a32eb94ddec1c87696bbbd28ad2e04739ee7037a016e6ba1f57
public key: 809669cf535af1ef9907032b3cf3a3c19b70763551bc71996c8b0dfcaf92878d

R_bytes:   282291f426d3cdcaa24fe4af1e81effd410fc6359029e13be4d65066c1eae16c
sig:       e5476ef2b158665d5c68eb349cee1c1f4248c5d48c8a3ab7122c30d7170b5308

msg_prime: b'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam eget vulputate dui, eget faucibus neque. Mauris semper interdum nulla, quis tristique dolor ultricies a. Sed placerat sagittis tristique. Nam leo sem, pharetra non tempor sed, aliquam ac arcu. Curabitur consequat nisl ac urna consequat tempus. Ut nec diam iaculis, dignissim nunc et, dignissim mauris. Nunc dapibus sit amet risus id rutrum. Nullam ex lorem, hendrerit sed felis sed, aliquet blandit ligula. Curabitur scelerisque leo ut sem aliquam accumsan ac vitae urna.'


In [21]:
# Verification value (`a % L`)
verify = bytes_to_clamped_scalar(H(skey)[:32])%L
print('verify:   ', verify.to_bytes(32, 'big').hex())

with open('data/spa.txt', 'r') as f:
    f.readline()
    # Recovered sha512(skey)[:32] % L, can be used for signing thanks to the magic of group 
    # arithmatic. However, sha512(skey)[32:] is missing, which is also needed to properly 
    # forge signatures. But verification will pass all the same.
    spa_key = int(parse_da(f.readline().strip()), 2)
    print('spa_key:  ', spa_key.to_bytes(32, 'big').hex())
    assert(spa_key == verify)
    
    # `spa_key` results in the same public key
    pkey2 = reverse_publickey(spa_key).to_bytes()
    assert(pkey2 == pkey)
    
    # `spa_key` results in a different signature, but verifies all the same
    tmp = forge_sign(msg, spa_key, pkey)
    R_bytes2  = tmp[:32]
    S2        = tmp[32:64]
    msg_orig2 = tmp[64:]
    assert(R_bytes2 != R_bytes)
    assert(S2 != S)
    assert(msg == msg_orig)
    assert(msg == raw.open(tmp, pkey2))

    f.readline()
    l = f.readline().strip()
    r = int(parse_da(l), 2)
    # Again, we cannot recover `b`, but as shown below it still results in a working signature.
    spa_key2 = reverse_sign(int.from_bytes(S, 'little'), r, pkey, msg)
    print('spa_key2: ', spa_key2.to_bytes(32, 'big').hex())
    assert(spa_key2 == verify)
    
    # `spa_key2` results in the same public key
    pkey2 = reverse_publickey(spa_key2).to_bytes()
    assert(pkey2 == pkey)
    
    tmp = forge_sign(msg, spa_key, pkey2)
    R_bytes2  = tmp[:32]
    S2        = tmp[32:64]
    msg_orig2 = tmp[64:]
    assert(R_bytes2 != R_bytes)
    assert(S2 != S)
    assert(msg == msg_orig)
    assert(msg == raw.open(tmp, pkey2))

verify:    0e5827d3653d2f5818b63dd46df96fc4c88263c19c48572e13e0c63e347d0105
spa_key:   0e5827d3653d2f5818b63dd46df96fc4c88263c19c48572e13e0c63e347d0105
spa_key2:  0e5827d3653d2f5818b63dd46df96fc4c88263c19c48572e13e0c63e347d0105
